Nom Etudiant 1: <font color='blue'>PARNICHTHANAKOM Pornpround</font>

# TME 2.2 : Projet Detection de motifs - Median String

Les motifs que nous cherchons dans les séquences d'ADN peuvent présenter certaines variations ou mutations. Nous allons développer différents algorithmes pour la recherche de motifs variables (**Partie A**), puis améliorer cette recherche en intégrant une liste étendue de motifs de référence et en développant des fonctions pour l'algorithme Median String (**Partie B**), avant d'appliquer ces approches aux données de ChIP-Seq de _C. glabrata_ (**Partie C**).

## Partie A : Recheche de pattern (motifs) en permettant des variations

1\. Comme dans le TME precedent, nous alons utiliser des données atificielles pour pouvoir tester les algorithmes. Reutilisez les séquences artificielles de taille `t = 41` et le motif de taille `k = 4` sauvegardés le TME précédente.

In [1]:
from TME import *

seqs_file = 'seqs_artificielles.txt'

f = open(seqs_file,"r")
tmp = f.readlines()
seqs = []
for line in tmp :
    seqs.append(line[:-1])
f.close()
        
for seq in seqs:
    print(seq)

motif_file = "motif.txt"
f = open(motif_file,"r")
motif = f.readline()
f.close()

print("\nLe motif géneré était :", motif)

ACATCCCTAGAAGGATGGTCTGTTCGGATCTGTTTCTTAGA
GTGCCACCTACACACTAACCCATTGGTGGTAACCACTCTGA
CTATTCGTGTAATAAGACAAGACGGCTGGATCGGCTTCTCT
CGAGGGATTACCTGAGAGGTAGTGTTGGCCGTCGATTTAAT
GAGCTTGTGTTTATGCAGTACGTCAGGAAATCTCAGCCATT
TCTTGCTATACTCTCGTCAAGCATCAGTAGCGATGGTGTCG
GTGGCGGGATCAGGGGCAGTTCCTCATCTACTGCGGCAGTG
GGGTCCGACAACCTACCGGTCGCACCAGTTTACCCGTTATA
CGTCAGAGAGCTACCAGTAATTAACATGCCAATCGGAGTGT
TCTTCGTCACGAGCCTTCTAGAGAAATTGATGCAGCTCGTC

Le motif géneré était : ACAT


2\. Jusqu'à maintenant, nous avons travaillé avec des motifs invariables. Cependant, dans les données biologiques, les motifs peuvent présenter des variations. Créez une fonction pour faire varier un motif selon un certain nombre de positions `v`.

In [2]:
import random

nuc = ('A', 'C', 'G', 'T')

# Pour obtenir le même résultat que dans le dévelopent de ce TME
#random.seed(seed)

v = 1 # nombre de positions variables dans le motif

def modifierMotif(motif:str, v:int, upper = False):
    """
    Modifie v positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée v: nombre de positions à varier, si 0 le motif sera invariable
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    
    motifM = motif
    pos = []

    for i in range(v) :
        idx = random.randint(0,len(motif)-1)
        while idx in pos :
            idx = random.randint(0,len(motif)-1)
        pos.append(idx)

        x = random.choice(nuc).lower()
        if upper : x.upper()

        motifM = motifM[:idx] + x + motifM[idx+1:]
    
    return motifM

print("Motif original : ", motif)

# Tester la fonction modifierMotif plusieurs fois
for i in range(0, 5):
    print(modifierMotif(motif, 1, upper = False))

Motif original :  ACAT
ACAc
ACAt
ACcT
tCAT
ACtT


3\. Modifiez la fonction `implantMotifs()` de la séance précédente afin d’ajouter la possibilité de varier le motif implanté. Cette fois-ci, considérez que toutes les séquences contiendront un motif (`f1 = 1`). En revanche, seule une proportion des séquences (`f2 = 0.4`) inclura un motif modifié. Pour faciliter leur identification, implantez les motifs en minuscules.

**BONUS :** Implantez un motif modifié dans une proportion de séquences légèrement inférieure (`f1 = 0.9`). Assurez-vous de ne pas ajouter un motif modifié si, par hasard, la fonction `modifierMotif()` génère le même motif que l’original.

In [3]:
f1 = 1
f2 = 0.4

# Pour obtenir le même résultat que dans le dévelopent de ce TME
#random.seed(seed)

def implantMotifsVar(motifs:str, sequences:list, v:int, f1 = 1, f2 = 0.4):
    """
    Insére un motif dans des positions aléatoires des séquences
    entrée motifs : motif qui va être implanté dans les séquences
    entrée séquences : liste de séquences
    entrée v: nombre de positions à varier, si 0 le motif sera invariable
    entrée f : fréquence d'implantation, si 1 toutes les séquences contiendront un motif
    sortie modified_sequences: liste de séquences ayant le motif implanté
    """
    
    modified_sequences = []
    modif = 0

    if f1 == 1 :
        modified_sequences = []
        for seq in sequences : 
            pos = random.randint(0,len(seq)-1)

            if modif/len(sequences) < f2 : 
                aux = modifierMotif(motifs,v)
                modif += 1 
            else : aux = motifs

            modified_sequences.append(seq[:pos].upper()+aux.lower()+seq[pos:].upper())
        return modified_sequences
    
    not_implanted = sequences
    implanted = []
    while len(implanted)/len(sequences) < f :
        seq = random.choice(not_implanted)
        not_implanted.remove(seq)
        pos = random.randint(0,len(seq)-1)

        imp = modifierMotif(motifs,v)
        if imp != motifs : implanted.append(seq[:pos].upper()+imp.lower()+seq[pos:].upper())
            
    return modified_sequences

seqs_implanted = implantMotifsVar(motif, seqs, v, f1, f2)

seqs_implanted

['ACATCCCTAGAAGGacaaATGGTCTGTTCGGATCTGTTTCTTAGA',
 'GTGCCACCTACACACTAACCCAaaatTTGGTGGTAACCACTCTGA',
 'CTATTCGTGTAATAAGACAAGACGGCTGGATCGacatGCTTCTCT',
 'CGAGGGATTACCTGAGAGGTAGTGTTGacatGCCGTCGATTTAAT',
 'GAGCTTGTGTTTATGCAacatGTACGTCAGGAAATCTCAGCCATT',
 'TCTTGCTATACTCTCGTCAAGCATCAGTAGCGATGGTGTCacatG',
 'GTGGCGGGATCAGGGGCAGTTCCTCATCacatTACTGCGGCAGTG',
 'GGGacatTCCGACAACCTACCGGTCGCACCAGTTTACCCGTTATA',
 'CGTCAGAGacatAGCTACCAGTAATTAACATGCCAATCGGAGTGT',
 'TCTTCGTCACGAGCCTTacatCTAGAGAAATTGATGCAGCTCGTC']

4\. Nous pouvons visualiser les motifs à l'aide des outils de WebLogo : https://weblogo.berkeley.edu/logo.cgi.
Extrayez les motifs implantés dans vos séquences artificielles et visualisez-les à l'aide de WebLogo.
Affichez le logo en utilisant **Markdown**.

In [4]:
motifs_extraits = []
for seq in seqs_implanted :
        tmp_motif = ""
        for x in seq :
                if x.islower() : tmp_motif += x
        motifs_extraits.append(tmp_motif)

print("Motif original : ", motif)

for motif_extrait in motifs_extraits:
        print(motif_extrait)

Motif original :  ACAT
acaa
aaat
acat
acat
acat
acat
acat
acat
acat
acat


![Logo](Logo_GCGT.png)

5\. Insérez le motif reverse complémentaire dans les séquences contenant déjà un motif implanté. Si vous avez réussi le **BONUS** de la question 3, vous pouvez insérer ce motif reverse complémentaire avec une fréquence légèrement plus faible, afin de simuler une présence inégale du motif varié reverse complémentaire.

In [5]:
# Importez vos fonctions
motif_rev = reversecompl(motif)

seqs_implanted_rev = implantMotifsVar(motif_rev,seqs_implanted,1,1,0.9)

seqs_implanted_rev

['ACATCCCTAGAAGGACAAATGGTCTGatatTTCGGATCTGTTTCTTAGA',
 'GTGCCACCTACACACTgtgtAACCCAAAATTTGGTGGTAACCACTCTGA',
 'CTATTCGTGTAATatttAAGACAAGACGGCTGGATCGACATGCTTCTCT',
 'CGAGGGATTACCTGAGAGGaggtTAGTGTTGACATGCCGTCGATTTAAT',
 'GAGCTTGTGTTTATGCAACATGatgtTACGTCAGGAAATCTCAGCCATT',
 'TCTTGCTATACTCTCGTCAAGCATCAGTAGCGATGGTGTCACttgtATG',
 'GTGGCGGGATCAGGGGCAGTTCCTCAatgtTCACATTACTGCGGCAGTG',
 'GGGACATTCCGACAACCTACCGGTCGCACCAGTTTACCCGttgtTTATA',
 'CGTCAGAGACATAGCTACCAGTAATTAACatgaATGCCAATCGGAGTGT',
 'TCTTCGTCACGAGCCTTACATCTAatgtGAGAAATTGATGCAGCTCGTC']

6\. Sauvegardez vos séquences artificielles implantées avec les motifs variables, ainsi que les motifs variés générés, afin de les réutiliser lors des séances suivantes.

In [6]:
seqs_impl_file = "seqs_artificielles_impl_var.txt"
motif_file = "motif_var.txt"
        
f = open(seqs_impl_file,"w") 
f.write("\n".join(seqs_implanted))
f.write("\n")
f.close()

f = open(motif_file,"w")
f.write("\n".join(motifs_extraits))
f.close()

## Partie B : Amellioration de la recherche de motifs et développement de l'algorithme Median String 

1\. Précédemment, nous avons recherché des motifs déjà présents dans nos données. Cependant, une meilleur option pourrait être de créer une liste de tous les motifs possibles d'une certaine taille, même si nous ne connaissons pas leur fréquence. Suivant cette logique, combien de motifs (k-mers) de taille `k = 4` pouvons-nous obtenir avec toutes les combinaisons possibles de nucléotides ? Effectuez le calcul avec les fonctions de **Python**. **BONUS :** vous pouvez utiliser **Markdown** en combinaison avec [LaTeX](https://fr.wikipedia.org/wiki/LaTeX), un système de composition de documents qui permet de représenter des expressions mathématiques (par example, $E = mc^2$) pour exprimer votre résultat.

In [7]:
# Calculer en Python
nt = ['A','T','C','G']
k = 4
nb_combin = len(nt)**k
nb_combin

256

**BONUS :** Résultat exprimé avec LaTeX

La quantité de k-mers possibles de taille 4 est donnée par: $$4^{4} = 256$$

2\. En utilisant les fonctions de la bibliothèque **itertools**, créez tous les k-mers possibles de taille `k = 4`.

In [8]:
k = 4

from itertools import product

# Générer tous les k-mers de taille k ayant de AAA... à TTT...
aux = list(product(nt,nt))
tmp = [a+b for a,b in aux]

aux = list(product(tmp,tmp))
allkmers_k4 = [a+b for a,b in aux]

# Visualiser les prémieres 10 k-mers
print(allkmers_k4[0:10])

['AAAA', 'AAAT', 'AAAC', 'AAAG', 'AATA', 'AATT', 'AATC', 'AATG', 'AACA', 'AACT']


3\. Éliminez les motifs peu complexes pour éviter les calculs inutiles (motifs ayant plusieurs fois une base répétée, comme **AAAAAAC** ou deux bases répétées comme **ACACACAC**). Réutilisez les fonctions déjà définies (`removeLowComplexeHomo()` et `removeLowComplexeHetero()`) de la séance précédente. Faites attention au type d'objet d'entrée pour chaque fonction. Réfléchissez aux meilleurs paramètres pour filtrer les k-mers selon leur taille. Combien de k-mers restent après avoir supprimé ceux qui sont peu complexes ?

In [9]:
# Importez vos fonctions

import re

# Réponse BONUS TME2.1 question 6
def removeLowComplexeHetero(motifs:list, n:int, variation = "yes"):
    """
    Enlève les motifs peu complexe ayant n fois un dinucléotide
    entrée motifs: liste de motifs, clé = motif, valeur = fréquence d'observation
    entrée n: taille de répétition de dinucléotide
    entrée variation : string, si "yes" permettre variation d'un nucléotide
    sortie motifsClean: liste de motifs sans les motifs peu complexe
    >>>removeLowComplexeHetero(['GGTTTGG', 'TGAGTTA', 'TGCCGTG', 'AGAGAGA', 'TCACCGA', 'TTGGTAT', 'AGGGTGG', 'TGGCTTA', 'AGAGTAG', 'GCCCCTC'], 3, variation = "yes")
    ['GGTTTGG', 'TGAGTTA', 'TGCCGTG', 'AGAGAGA', 'TCACCGA', 'TTGGTAT', 'TGGCTTA', 'AGAGTAG']
    """
    
    motifsClean = []
    trouve = []

    # Créer une liste de dinucléotides sans mono dinucléotides
    all_di_nuc = [nuc_1 + nuc_2 for nuc_1 in nuc for nuc_2 in nuc if nuc_1 != nuc_2]

    # Pour chaque motif de la liste
    for motif in motifs:
        if variation == "yes":
            # Permettre une varation
            trouve = [nuc_di for nuc_di in all_di_nuc if len(re.findall(re.compile("(?=("+ nuc_di +"))"), motif)) >= n]

        else:
            # Pas permettre de varations
            trouve = [nuc_di for nuc_di in all_di_nuc if len(re.findall(re.compile("(?=("+ nuc_di +")"+ nuc_di[0] +")"), motif)) >= n]

        # Après avoir evalué tous les dinucléotides, évaluer si on a trouvé de dinucleotides
        if len(trouve) == 0:
            # Ajouter juste les motifs que n'ont pas eu des dinucléotides trouves
            motifsClean.append(motif)
        # Vider la liste
        trouve = []

    return motifsClean

# Liste originale
print(len(allkmers_k4))

# Sans homo
allkmers_k4 = removeLowComplexeHomo(allkmers_k4,4)

print(len(allkmers_k4))

# Sans hetero
allkmers_k4 = removeLowComplexeHetero(allkmers_k4,1,"no")

print(len(allkmers_k4))

256
252
168


4\. Implémentez l'algorithme _"Median String Search"_ pour chercher des motifs de taille variable. Tout d'abord, créez une fonction pour calculer la distance de Hamming.

In [10]:
def hamDistance(str1:str, str2:str):
    """
    Calcul la distance de Hamming entre deux chaînes de caractères
    entrée str1: chaîne de caractères
    entrée str2: chaîne de caractères
    sortie distance: distance de Hamming
    >>>hamDistance("TTGGTAT", "TTGCTAA")
    2
    """
    
    distance = 0
    for i in range(len(str1)) :
        if str1[i] != str2 [i] : distance += 1
        
    return distance

hamDistance("TTGGTAT", "TTGCTAA")

2

5\. Postérieurement, créez une fonction permettant d'obtenir la distance totale minimale entre un motif et un ensemble de séquences.

In [11]:
# Attention à ne pas comparer des séquences en majuscules avec des minuscules.

def totalDistance(motif:str, sequences, k):
    """
    Calcul la totalDistance
    entrée motif: motif à comparer, chaîne de caractères
    entrée sequences: liste de séquences
    entrée k: taille du motif
    sortie total_distance: somme de distance de hamming minimal
    """
    
    total_distance = 0
    for seq in sequences :
        dist = []
        for i in range(len(seq)-k+1) :
            aux = seq[i:i+k].upper()
            dist.append(hamDistance(motif,aux))
        total_distance += min(dist)

    return total_distance

totalDistance(motif, seqs_implanted, k)

1

6\. Finalmente, implémentez l'algorithme complet pour tester un liste de motifs et produire un dictionnaire contenant toutes les distances calculées à partir de tous les motifs testés. Trouvez le motif qui donne la distance minimale avec `getTopMotifs()`. Faites attention à ordonner le dictionnaire  dans le bon sens (croissant ou decroissant).

In [12]:
def medianStringSearch(sequences, kmersV):
    """
    Implement l'algorithme MedianStringSearch
    entrée séquences : liste de séquences
    entrée kmersV: Liste de Kmers à chercher
    sortie motif_dist_dict: un dictionnaire contenant les motifs et leurs distances
    """
    
    motif_dist_dict = {k:totalDistance(k,sequences,len(kmersV[0])) for k in kmersV}

    return getTopMotifs(motif_dist_dict,len(motif_dist_dict),False)

In [13]:
# Importez vos fonctions

f = open("seqs_artificielles_impl_var.txt","r")
seqs_implanted_motifs = []
tmp = f.readlines()
for line in tmp :
    seqs_implanted_motifs.append(line[:-1])
f.close()

kmersV = []
aux = list(product(nt,nt))
tmp = [a+b for a,b in aux]

aux = list(product(tmp,tmp))
kmersV = [a+b for a,b in aux]

seqs_implanted_motifs_top = medianStringSearch(seqs_implanted_motifs,kmersV)

print(seqs_implanted_motifs_top)

{'ACAT': 1, 'ATGC': 5, 'GACA': 5, 'AAAT': 6, 'TCAG': 6, 'CATC': 6, 'CATG': 6, 'CGTC': 6, 'GTCA': 6, 'GTGT': 6, 'ATTA': 7, 'ATCT': 7, 'ACAA': 7, 'ACCA': 7, 'ACCT': 7, 'AGTG': 7, 'AGCT': 7, 'TAAT': 7, 'TTAC': 7, 'TTTA': 7, 'TTCG': 7, 'TCAC': 7, 'TCGT': 7, 'TGCC': 7, 'CATT': 7, 'CACA': 7, 'CAGT': 7, 'CTAC': 7, 'GCCA': 7, 'GGAT': 7, 'GGGA': 7, 'AATT': 8, 'AACA': 8, 'ATTC': 8, 'ATCA': 8, 'AGTA': 8, 'TACA': 8, 'TACT': 8, 'TACC': 8, 'TTAA': 8, 'TCTG': 8, 'TCGG': 8, 'TGTT': 8, 'TGGT': 8, 'CAAG': 8, 'CTTA': 8, 'CTTC': 8, 'CTCA': 8, 'CTCG': 8, 'CCAG': 8, 'CGAC': 8, 'GATC': 8, 'GAGA': 8, 'GTAG': 8, 'GTTT': 8, 'GTCG': 8, 'GGTC': 8, 'GGTG': 8, 'AATA': 9, 'AATC': 9, 'ATAC': 9, 'ATTT': 9, 'ATTG': 9, 'ATCC': 9, 'ATCG': 9, 'ATGG': 9, 'ACAC': 9, 'ACGT': 9, 'AGAA': 9, 'AGAC': 9, 'AGTT': 9, 'AGCC': 9, 'AGGA': 9, 'TAAC': 9, 'TATA': 9, 'TATT': 9, 'TACG': 9, 'TAGT': 9, 'TAGC': 9, 'TTAT': 9, 'TTAG': 9, 'TTCC': 9, 'TTGA': 9, 'TTGC': 9, 'TCAA': 9, 'TCTT': 9, 'TCTC': 9, 'TCCC': 9, 'TCCG': 9, 'TGAT': 9, 'TGTA': 9

Avez-vous trouvé le motif implanté ? Parfois, le motif implanté n'est pas le plus fréquent. Vous pouvez explorer le dictionnaire obtenu avec la fonction  `searchGivenMotif()`.

In [14]:
# Importez vos fonctions

searchGivenMotif(seqs_implanted_motifs_top, "ACAT", False)

(1, 1)

7\. Rassemblez les résultats de l'algoritme MedianStringSearch sur les séquences du brin complementaire en réutilisant la fonction `gatherRevCompMotifs()`. Combien de motifs restent après avoir fusionné le dictionnaire ? Affichez le nouveau résultat.

In [15]:
# Importez vos fonctions

print("Avant rassemblage :")

print(len(seqs_implanted_motifs_top))

seqs_implanted_motifs_gat = gatherRevCompMotifs(seqs_implanted_motifs_top) 

print("Après rassemblage :")

print(len(seqs_implanted_motifs_gat))

seqs_implanted_motifs_gat_top = getTopMotifs(seqs_implanted_motifs_gat, 10, False)

print(seqs_implanted_motifs_gat_top)

Avant rassemblage :
256
Après rassemblage :
136
{'CATG': (6, 6), 'ATTA': (7, 7), 'AGCT': (7, 7), 'ACCA': (8, 7), 'CTAC': (8, 7), 'AATT': (8, 8), 'AACA': (8, 8), 'AGTA': (8, 8), 'TTAA': (8, 8), 'CGAC': (8, 8)}


## Partie C : Recherche de motifs variable sur vos données

1\. Utilisez le fichier "Sequence_by_Peaks_##.fasta", contenant les régions de peaks identifiées par ChIP-Seq, où se trouve probablement le facteur de transcription recherché. Appliquez l'algorithme Median String pour détecter les motifs Il faut bien evidement enlever les motifs peu complexe. Créez et testez different tailles de motifs. N'oubliez pas de rassembler le dictionnaire obtenu.

In [16]:
k = 7 # tester d'autres tailles k = 6, k = 8, etc.

m = 5

n = 2

peaks = readFasta("Sequence_by_Peaks_1.fasta")

allkmers_k7 = ["".join(kmer) for kmer in list(product('ACGT', repeat=k))]

# Liste originel
print(len(allkmers_k7))

allkmers_k7 = removeLowComplexeHomo(allkmers_k7, m)

# Sans homo
print(len(allkmers_k7))

allkmers_k7 = removeLowComplexeHetero(allkmers_k7, n)

# Sans hetero
print(len(allkmers_k7))

16384
15540
9708


In [17]:
peaks_motifs = gatherRevCompMotifs(medianStringSearch(peaks, allkmers_k7))

peaks_motifs_top = getTopMotifs(peaks_motifs, 10, False)

print("Top motifs :\n", peaks_motifs_top)

Top motifs :
 {'ATTTTAA': (158, 157), 'TGAAATA': (159, 157), 'AAAATTT': (159, 157), 'TGCAAAA': (160, 159), 'ATTTGAA': (161, 153), 'TAAATTT': (162, 157), 'TTCAAAA': (164, 155), 'AGAAATT': (164, 156), 'TTGAAAA': (164, 158), 'TTTCAAA': (164, 158)}


2\. Parfois, la TATA box, un type de motif général présent dans les promoteurs de gènes, dépasse le seuil établi pour filtrer les homopolymères et hétéropolymères. Cependant, ces TATA box constituent des motifs généraux plutôt que spécifiques. Une approche "naïve" pour les éliminer consiste à filtrer les motifs contenant une forte proportion de nucléotides T et A. Créez une fonction qui supprime les motifs enrichis en T et A, en veillant à ne les retirer que si les deux nucléotides sont présents.

In [20]:
p = 0.7

def removeTARich(motifs:list, p:int):
    """
    Enlève les motifs contenant > p de T et A en combination
    entrée motifs: liste de motifs
    entrée p: proportion de nucleótides T et A
    sortie motifsClean: liste de motifs sans les motifs TA rich
    """

    motifsClean = []

    for m in motifs :
        cpt = 0
        a,t = False,False
        for x in m :
            if x == 'A' :
                cpt += 1
                a = True
            if x == 'T' :
                cpt += 1
                t = True
        if not (cpt/len(m) > p) and (a and t) : motifsClean.append(m)

    return motifsClean

test_motifs_TA = ['ATTTTCA', 'TAATTTA', 'AAATTAT', 'TCTACGA', 'ACGCATT', 'ACAAGGT', 'GAAATTA', 'TTGTTGT', 'TATAGCA', 'TAGCCTT']

removeTARich(test_motifs_TA, p)

['TCTACGA', 'ACGCATT', 'ACAAGGT', 'TAGCCTT']

3\. Retestez après avoir enlevé les motifs enrichis en TA.

In [21]:
allkmers_k7 = removeTARich(allkmers_k7, p)

peaks_motifs = gatherRevCompMotifs(medianStringSearch(peaks, allkmers_k7))

peaks_motifs_top = getTopMotifs(peaks_motifs, 10, False)

print("Top motifs :\n", peaks_motifs_top)

Top motifs :
 {'CTGCAAA': (182, 182), 'GTGCAAT': (183, 181), 'CTTGCAA': (183, 182), 'TTTGCAC': (184, 181), 'CCATTTC': (184, 181), 'CAAATGG': (186, 182), 'TGGCAAA': (186, 183), 'TCTTGCA': (186, 185), 'CATTTCC': (186, 186), 'AGTTGCA': (187, 180)}


**NOTES :** Pensez à developer une fonction pour créer un liste de motifs de manière automatique. Aussi, appliquez les stratégies appris dans ce TME à l'algorithme de la Hash Table :

* Utilisation d'une liste étendue de motifs de référence.
* Suppression des séquences enrichies en T et A.